In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyMELTScalc as M
import sys

sys.path.append(r"H:\My Drive\2022\GitHub\pyMELTScalc\MELTS")

# MELTS tutorial Q1 & 2 - liquidus, equilibrate, and crystallisation calculations

This notebook will take you through some simple calculations used to show how we might set up a crystallisation calculation for a high-silica rhyolite. The composition used is from the Oldest Toba Tuff. 

Above you should have loaded in the common Python packages, and told Python where to find the underlying alphaMELTS for Python code (in the sys.path.append(r' ') copy the location of the MELTS files on your PC).

First, lets set a starting composition. To do this we simply create a Python dictionary with the amount of each oxide.

In [2]:
comp = {'SiO2_Liq': 74.39,
        'TiO2_Liq': 0.18,
        'Al2O3_Liq': 13.55,
        'FeOt_Liq': 1.30,
        'MgO_Liq': 0.50,
        'CaO_Liq': 1.43,
        'Na2O_Liq': 3.36,
        'K2O_Liq': 5.09}

### Q1

- **Using a pressure of 1750 bars (175 MPa) and an oxygen fugacity constrained at the Ni-NiO buffer, and a melt H$_2$O concentration of 4 wt%, calculate the liquidus temperature**

This can be done using the "findLiq_multi" function from pyMELTScalc.

In [3]:
import os
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [4]:
Results = M.findLiq_multi(Model = "MELTSv1.0.2", 
                        bulk = comp,
                        P_bar = 1750,
                        H2O_Liq = 4,
                        fO2_buffer = "NNO")    

  0%|          | 0/1 [00:00<?, ?it/s]

This function will return a Dictionary containing the liquidus temperature (in Celsius), the liquidus phase, and the normalized melt composition (including volatile components).

In [5]:
Results

{'T_Liq': 860.5499999999996,
 'liquidus_phase': 'orthopyroxene1',
 'fluid_saturated': 'No',
 'SiO2': 71.6620381025268,
 'TiO2': 0.1735134361588442,
 'Al2O3': 13.061377001621128,
 'Fe2O3': 0.33688102366157663,
 'Cr2O3': 0.0,
 'FeO': 0.9285092824402547,
 'MnO': 0.0,
 'MgO': 0.48153035635052815,
 'CaO': 1.3771768738589403,
 'Na2O': 3.2358842516123345,
 'K2O': 4.901979611371476,
 'P2O5': 0.0,
 'H2O': 3.8522436150133244,
 'CO2': 0.0}

-
-
-
-



- **Keeping all other settings the same, what is the liquidus temperature of the magma when it is water saturated?**

- **What is the liquidus phase at the water-saturated liquidus?**

Try increasing the H$_2$O content until the "fluid_saturated" value changes to "Yes".

- **Using your results, what is the solubility of water at the liquidus (according to the rhyoliteMELTSv1.0.2 model)?**

In [6]:
Results = M.findLiq_multi(Model = "MELTSv1.0.2", 
                            bulk = comp,
                            P_bar = 1750,
                            H2O_Liq = 8,
                            fO2_buffer = "NNO")

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
Results

{'T_Liq': 844.9499999999992,
 'liquidus_phase': 'orthopyroxene1',
 'fluid_saturated': 'Yes',
 'SiO2': 70.95370520629845,
 'TiO2': 0.17179426420736071,
 'Al2O3': 12.932131584456753,
 'Fe2O3': 0.33813616162337573,
 'Cr2O3': 0.0,
 'FeO': 0.9305149718461792,
 'MnO': 0.0,
 'MgO': 0.47676246874501194,
 'CaO': 1.3635415406840388,
 'Na2O': 3.2038479281708523,
 'K2O': 4.8534513409840825,
 'P2O5': 0.0,
 'H2O': 4.80351966268605,
 'CO2': 0.0}

-
-
-
-


Note that the pyMELTScalc output is normalised to a hydrous total of 100 wt%. By default, MELTS treats the input composition as grams of the different oxide values (e.g., grams of SiO$_2$, grams of TiO$_2$ etc.). How many grams of H$_2$O are required for the magma to be water saturated at the liquidus at 1750 bars?

This can be determined by multiplying the H$_2$O content in wt% by the ratio of the inputted SiO$_2$ content (in grams) and the outputted SiO$_2$ content (in wt%).

In [8]:
H2O_gram = Results['H2O']*comp['SiO2_Liq']/Results['SiO2']
H2O_gram

5.036154583728423

- **Using the M.equilibrate_multi function, determine the mass percent of melt remaining int he system at 765$^o$C for a melt that is water-saturated (but contains no exsolved water) at the liquidus?**

In [9]:
Eq_Results = M.equilibrate_multi(Model = "MELTSv1.0.2", 
                            bulk = comp, 
                            T_C = 765, 
                            P_bar = 1750, 
                            H2O_Liq = H2O_gram, 
                            fO2_buffer = "NNO")

  0%|          | 0/1 [00:00<?, ?it/s]

The results are all stored in a Dictionary called Eq_Results. Within this dictionary there is a DataFrame called 'Mass' (i.e., Eq_Results['Mass']) that contains  the mass of each phase at this pressure and temperature.

In [10]:
Eq_Results['Mass']

,liquid1,orthopyroxene1,plagioclase1,spinel1,water1
0,94.700574,1.785977,7.737928,0.015368,0.62159


- **What is the Anorthite content of the plagioclase formed at this temperature?**

The composition of any phase can be accessed in the Eq_Results Dictionary by calling the DataFrame with the phase name followed by 1. i.e., for plagioclase compositions you would type Eq_Results['plagioclase1'].

In [11]:
Eq_Results['plagioclase1']

,SiO2_Plag,TiO2_Plag,Al2O3_Plag,Cr2O3_Plag,Fe2O3_Plag,FeO_Plag,FeOt_Plag,MnO_Plag,MgO_Plag,CaO_Plag,Na2O_Plag,K2O_Plag,P2O5_Plag,H2O_Plag,CO2_Plag,Fe3Fet_Plag
0,62.399416,0.0,23.439,0.0,0.0,0.0,0.0,0.0,0.0,4.777365,8.167063,1.217155,0.0,0.0,0.0,NaN


pyMELTScalc doesn't automatically give you the mineral proportions in terms of their end-member components. However, one of the great things about pyMELTScalc is that it's specifically designed to integrate with other Python packages. For example, here we can use Thermobar to calculate the Plagioclase cation fractions and end-member components without having to modify our output at all!

In [14]:
import Thermobar as pt
Plag = pt.calculate_cat_fractions_plagioclase(plag_comps = Eq_Results['plagioclase1'])
Plag['An_Plag']

0    0.227435
Name: An_Plag, dtype: float64

### Q2

- **Using the same conditions and composition as above, run as isobaric cooling sequence (for now we will assume equilibrium crystallisation) with fO$_2$ constained at the Ni-NiO buffer. Start this simulation at the "wet liquidus" and continue until the system is more than 90% crystals!**

For this, we will use a funcion termed M.isobaric_crystallisation

In this function we need to provide the pressure and fO$_2$ constraints as well as several other bits of information. Read the comments below to find out what each variable does:

In [15]:
Cryst_Results = M.isobaric_crystallisation(Model = "MELTSv1.0.2", # the version of MELTS to be used in the calculation
               bulk = comp, # starting composition as used previously
               T_start_C = 845, # temperature to start the calculation - use the wet liquidus temperature calculated above
               T_end_C = 750, # end temperature, likely won't be reached in this model but needed as a place holder
               dt_C = 0.5, # temperature intervals
               P_bar = 1750, # pressure (bars)
               H2O_Liq = 8, # grams of H2O in the system
               fO2_buffer = "NNO", # fO2 buffer
               fluid_sat = True, # if True the system will start at the wet liquidus, but remove any excess fluid prior to crystallisation
               Crystallinity_limit = 0.90) # Using this command tells the code to finish the model when the crystal fraction passes above the specified value

Some questions to think about once this model has finished running.

- **At what temperature does quartz saturate?**

- **What is the SiO$_2$ content at quartz saturation?**

- **At what temperature does the melt abundance (in wt%) go below 10%?**

- **What would we traditionally call the temperature at which the melt % drops below 10%?**

All outputs included within pyMELTScalc are saved in a pandas DataFrame called 'All'. We can simply select the parameters that we are interested in investigating using the following code:

In [16]:
Cryst_Results['All'].loc[:, ['T_C', 'SiO2_Liq', 'mass_Liq', 'mass_Qtz']]

,T_C,SiO2_Liq,mass_Liq,mass_Qtz,mass_water1
0,845.0,70.932401,97.286177,0.000000,0.000180
1,844.5,70.955491,97.175004,0.000000,0.004203
2,844.0,70.957279,97.165776,0.000000,0.004286
3,843.5,70.959069,97.156556,0.000000,0.004372
4,843.0,70.960862,97.147343,0.000000,0.004459
...,...,...,...,...,...
178,756.0,73.099004,83.757277,0.000000,0.816030
179,755.5,73.141862,83.464380,0.000000,0.831381
180,755.0,73.144011,58.634846,8.397989,1.983146
181,754.5,73.329033,10.064439,25.238920,4.213264
